In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch transformers datasets pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.0 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
import torch
from datasets import load_dataset
import os

In [4]:
dataset= load_dataset("community-datasets/offenseval_dravidian", "kannada")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6217 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/777 [00:00<?, ? examples/s]

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6217
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 777
    })
})


In [16]:
train_df=pd.DataFrame(dataset['train'])
val_df=pd.DataFrame(dataset['validation'])

In [18]:
df = pd.concat([train_df, val_df], ignore_index=True)

In [19]:
df['label'].value_counts()

label
0    3970
5    1713
2     553
3     374
1     245
4     139
Name: count, dtype: int64

In [20]:
df['label'] = df['label'].replace({2: 1, 3: 1, 4: 1})

In [21]:
df['label'].value_counts()

label
0    3970
5    1713
1    1311
Name: count, dtype: int64

In [22]:
df = df[df['label'] != 5]

In [41]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame with columns 'text' and 'label'
# Initial class distribution
print(df['label'].value_counts())

# Separate features and target variable
X = df['text'].values.reshape(-1, 1)  # Reshape text column to 2D array
y = df['label']

# Oversample the minority class (label 1)
oversampler = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversampler.fit_resample(X, y)

# Combine the oversampled data back into a DataFrame
df_over = pd.DataFrame(X_over, columns=['text'])
df_over['label'] = y_over

# Display the class distribution after oversampling
print(df_over['label'].value_counts())

# Undersample the majority class (label 0)
undersampler = RandomUnderSampler(sampling_strategy='not minority')
X_under, y_under = undersampler.fit_resample(df_over['text'].values.reshape(-1, 1), df_over['label'])

# Combine the undersampled data back into a DataFrame
df_resampled = pd.DataFrame(X_under, columns=['text'])
df_resampled['label'] = y_under

# Display the final class distribution
print(df_resampled['label'].value_counts())

# Display the first few rows of the resampled DataFrame
print(df_resampled.head())

# Split the resampled data into training and validation sets
train_df, val_df = train_test_split(df_resampled, test_size=0.2, stratify=df_resampled['label'])

# Display the class distribution in the training and validation sets
print(train_df['label'].value_counts())
print(val_df['label'].value_counts())


label
0    3970
1    1311
Name: count, dtype: int64
label
0    3970
1    3970
Name: count, dtype: int64
label
0    3970
1    3970
Name: count, dtype: int64
                                                text  label
0  Tik tok alli jagala madtidralla adra baggenu o...      0
1                          Movie rerelease madi plss      0
2  Amazon prime alli bittidira....yella manele no...      0
3  Guru sure news nanu tik tok dawn lod madeda ya...      0
4  ಸುದೀಪ್ ಸರ್ ಅಂಡ್ ದರ್ಶನ್ ಸರ್ ಅವರಿಗೆ ಇರೋ ಫ್ಯಾನ್ಸ್...      0
label
1    3176
0    3176
Name: count, dtype: int64
label
0    794
1    794
Name: count, dtype: int64


In [42]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

In [43]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [44]:
MAX_LEN = 128
BATCH_SIZE = 16

import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Number of devices:", torch.cuda.device_count())
    # Set device to the first available GPU
    device = torch.device("cuda:0")
    print("Current device:", device)
else:
    print("CUDA is not available.")
device = torch.device("cuda")
train_dataset = TextClassificationDataset(
    texts=train_df['text'].to_numpy(),
    labels=train_df['label'].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

val_dataset = TextClassificationDataset(
    texts=val_df['text'].to_numpy(),
    labels=val_df['label'].to_numpy(),
    tokenizer=tokenizer,
    max_len=MAX_LEN
)

def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)


CUDA is available. Number of devices: 1
Current device: cuda:0


In [45]:
import torch

# Check for available devices
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=2)

# Move model to device
model.to(device)

Using device: cuda


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [46]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [47]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [48]:
trainer.train()


Step,Training Loss
10,0.696800
20,0.699000
30,0.692100
40,0.685600
50,0.680700
60,0.677500
70,0.656100
80,0.667100
90,0.641300
100,0.639500


TrainOutput(global_step=1191, training_loss=0.35372127506625467, metrics={'train_runtime': 280.7017, 'train_samples_per_second': 67.887, 'train_steps_per_second': 4.243, 'total_flos': 631074687197184.0, 'train_loss': 0.35372127506625467, 'epoch': 3.0})

In [49]:
eval_result = trainer.evaluate()
print(f"Evaluation results: {eval_result}")

Evaluation results: {'eval_loss': 0.364609032869339, 'eval_runtime': 5.9663, 'eval_samples_per_second': 266.16, 'eval_steps_per_second': 16.761, 'epoch': 3.0}


In [50]:
def predict(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        _, prediction = torch.max(outputs.logits, dim=1)
    return prediction.item()

In [51]:
new_text = "@Troll Stupid Fans heno helkodthidallla adunna helkodo munche neenu kaltho henge irbeku antha suvar gandu anthidde ivga ninge baididke thika huritha"
label = predict(new_text)
print(f"Predicted label: {label}")

Predicted label: 1


In [52]:
new_text = "Kannada film industry avaru sir solpa nodi rii yen madta ididra inta ashvath sir avara maga ondu sanni veshe dalli kalsa Kodi"
label = predict(new_text)
print(f"Predicted label: {label}")

Predicted label: 0


In [53]:
import torch

# Save the model's state_dict to a .pt file
torch.save(model.state_dict(), 'kanmodel_state_dict.pt')

# Save the entire model (including the architecture)
torch.save(model, 'kanmodel.pt')

# Save the tokenizer
tokenizer.save_pretrained('kanmodel_tokenizer')

('kanmodel_tokenizer/tokenizer_config.json',
 'kanmodel_tokenizer/special_tokens_map.json',
 'kanmodel_tokenizer/vocab.txt',
 'kanmodel_tokenizer/added_tokens.json')

In [54]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the state_dict
#model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6)
#model.load_state_dict(torch.load('/content/drive/MyDrive/models/tammodel_state_dict.pt'))

# Alternatively, load the entire model
model = torch.load('/content/drive/MyDrive/models/kanmodel.pt')

# Set the model to evaluation mode
model.eval()

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('/content/drive/MyDrive/models/kanmodel_tokenizer')

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

import torch
from transformers import DistilBertTokenizer

# Function to make predictions
def predict(texts, model, tokenizer, device, max_len=128):
    model.to(device)
    model.eval()

    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)
    return predictions.cpu()  # Move predictions back to CPU

# Example usage
texts = [
    "@Troll Stupid Fans heno helkodthidallla adunna helkodo munche neenu kaltho henge irbeku antha suvar gandu anthidde ivga ninge baididke thika huritha",
    "Kannada film industry avaru sir solpa nodi rii yen madta ididra inta ashvath sir avara maga ondu sanni veshe dalli kalsa Kodi"
]

predictions = predict(texts, model, tokenizer, device)
print("Predictions:", predictions.numpy())

Predictions: [1 0]
